In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
import base64
from PIL import Image, ImageTk
from tkinter import Tk, PhotoImage
import io
import tempfile
import sys
import os
import subprocess
import pkg_resources
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import webbrowser
from cryptography.fernet import Fernet

icon_path = 'logo.png'

# Replace 'base64_icon' with your actual base64-encoded image

# Convert the base64 string to bytes
icon_data = base64.b64decode(base64_icon)

# Create the main window and buttons
root = tk.Tk()
# Create a PhotoImage object from the bytes
icon_image = PhotoImage(data=icon_data)
root.iconphoto(True, icon_image)
root.title("Folder Encrypt Decrpyt")
root.geometry("600x500")
window_width = 600
window_height = 500
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = int((screen_width / 2) - (window_width / 2))
y = int((screen_height / 2) - (window_height / 2))
root.geometry(f"{window_width}x{window_height}+{x}+{y}")
root.configure(bg='black')
button_frame = tk.Frame(root, bg="grey")
button_image = None

def project_info():
    html_code = """
    </table>
    """

    # Save the HTML content to a temporary file
    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.html') as temp_file:
        temp_file.write(html_code)
        temp_file_path = temp_file.name

    # Open the temporary HTML file in the default web browser
    webbrowser.open('file://' + os.path.realpath(temp_file_path))


# Define the function to be executed when the first button is clicked
def button1_clicked():
    global folder_window
    folder_window = tk.Toplevel(root)
    folder_window.geometry("560x200")
    folder_window.title("Select Folder")

    # create a Label widget for the folder path
    folderpath_label = tk.Label(folder_window, text="folder Path")
    email_label = tk.Label(folder_window,text="Sender Email")
    smtp_label = tk.Label(folder_window,text="SMTP Password")
    receiver_label = tk.Label(folder_window,text="Receiver Email")
    # create an Entry widget for the folder path
    folderpath_entry = tk.Entry(folder_window, width=60)
    email_entry = tk.Entry(folder_window, width=60)
    smtp_entry = tk.Entry(folder_window, width=60,show="*")
    receiver_entry = tk.Entry(folder_window, width=60)
    
    # create a function to handle button clicks
    def browse_folder():
        # display the folder dialog and get the selected folder
        folder_path = filedialog.askdirectory()

        # update the Entry widget with the selected folder path
        folderpath_entry.delete(0, tk.END)
        folderpath_entry.insert(0, folder_path)

    global button_image

    # create a Button widget for browsing folders
    browse_button = tk.Button(folder_window,text="Browse Folders",command=browse_folder)



    def encrypt_folder():
        folder_path = folderpath_entry.get()
        sender_email = email_entry.get()
        receiver_email = receiver_entry.get()
        smtp_password = smtp_entry.get()
        if folder_path == '':
            messagebox.showerror("Error", "Please select a folder to encrypt.")
            return
        if not os.path.exists(folder_path):
            messagebox.showerror("Error", "Invalid folder path.")
            return
        # generate a key for encryption
        key = Fernet.generate_key()

        # create a Fernet object with the key
        fernet = Fernet(key)

        # iterate over all files in the directory
        for filename in os.listdir(folder_path):
            filepath = os.path.join(folder_path, filename)
            if os.path.isfile(filepath):
                # read the contents of the file
                with open(filepath, 'rb') as file:
                    data = file.read()

                # encrypt the data using Fernet
                encrypted_data = fernet.encrypt(data)

                # write the encrypted data back to the file
                with open(filepath, 'wb') as file:
                    file.write(encrypted_data)

        # save the key to a mail for decryption later
        sender_email = sender_email
        receiver_email = receiver_email
        subject = 'The Key for Encrypted Folder'
        message = 'The Key for Encrypted Folder ' + filepath + ' is:\n'+str(key) 
        
        # Set up SMTP server configuration
        smtp_server = "smtp.gmail.com"
        smtp_port = 587
        smtp_username = sender_email
        smtp_password = smtp_password

        # Create a multipart message object
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject

        #Add a text message to the email
        msg.attach(MIMEText(message, 'plain'))

        #Connect to the SMTP server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        try:
            server.login(smtp_username, smtp_password)
        except smtplib.SMTPAuthenticationError:
            messagebox.showerror("Error","Wrong SMTP Password")
            return

        server.send_message(msg)
        server.quit()
        messagebox.showinfo("Info", "Folder encrypted successfully.")

    encrypt_button = tk.Button(folder_window,text="Encrypt Folder", font=("Arial", 14, "bold"),bg="red", fg="white",command=encrypt_folder)

    # arrange the widgets using grid
    folderpath_label.grid(row=0, column=0, sticky=tk.E)
    folderpath_entry.grid(row=0, column=1)
    browse_button.grid(row=0, column=2)
    email_label.grid(row=1,column=0)
    email_entry.grid(row=1,column=1)
    smtp_label.grid(row=2,column=0)
    smtp_entry.grid(row=2,column=1)
    receiver_label.grid(row=3,column=0)
    receiver_entry.grid(row=3,column=1)
    encrypt_button.grid(row=4, column=1)

    # center the widgets
    folder_window.grid_rowconfigure(0, weight=0)
    folder_window.grid_columnconfigure(1, weight=10)
    folderpath_label.grid(padx=10,sticky="e")
    folderpath_entry.grid(pady=10,sticky="w")
    browse_button.grid(padx=10,pady=2, sticky="w")
    email_label.grid(padx=10,sticky="w")
    email_entry.grid(pady=10,sticky="w")
    smtp_label.grid(padx=10,sticky="w")
    smtp_entry.grid(pady=10,sticky="w")
    receiver_label.grid(padx=10,sticky="w")
    receiver_entry.grid(pady=10,sticky="w")
    encrypt_button.grid(padx=100,sticky="w")

    # prevent the folder window from being destroyed
    folder_window.grab_set()


# Define the function to be executed when the second button is clicked
def button2_clicked():
    global folder_window
    folder_window = tk.Toplevel(root)
    folder_window.geometry("560x200")
    folder_window.title("Select Image")

    # create a Label widget for the folder path
    folderpath_label = tk.Label(folder_window, text="Folder Path")
    password_label = tk.Label(folder_window,text="Password")
    # create an Entry widget for the folder path
    folderpath_entry = tk.Entry(folder_window, width=60)
    password_entry = tk.Entry(folder_window, width=60)
    
    # create a function to handle button clicks
    def browse_folder():
        # display the folder dialog and get the selected folder
        folder_path = filedialog.askdirectory()

        # update the Entry widget with the selected folder path
        folderpath_entry.delete(0, tk.END)
        folderpath_entry.insert(0, folder_path)

    global button_image

    # create a Button widget for browsing folders
    browse_button = tk.Button(folder_window,text="Browse Folders",command=browse_folder)


    def decrypt_folder():
        folder_path = folderpath_entry.get()
        password = password_entry.get()
        if folder_path == '':
            messagebox.showerror("Error", "Please select a folder to encrypt.")
            return
        if not os.path.exists(folder_path):
            messagebox.showerror("Error", "Invalid folder path.")
            return
        if password == '':
            messagebox.showerror("Error", "Please Enter a password")
            loading_window.destroy()
            return
        try:
            key = bytes(password.encode())
            key = key[2:-1]
            try:
                # create a Fernet object with the key
                fernet = Fernet(key)

                # iterate over all files in the directory
                for filename in os.listdir(folder_path):
                    filepath = os.path.join(folder_path, filename)
                    if os.path.isfile(filepath):
                        # read the encrypted contents of the file
                        with open(filepath, 'rb') as file:
                            encrypted_data = file.read()

                        # decrypt the data using Fernet
                        decrypted_data = fernet.decrypt(encrypted_data)

                        # write the decrypted data back to the file
                        with open(filepath, 'wb') as file:
                            file.write(decrypted_data)
                messagebox.showinfo("Info", "Folder Decrypted Successfully.")
            except InvalidToken:
                messagebox.showerror("Error", "Invalid Key. Decryption failed")
                return
        except:
             messagebox.showinfo("Info", "Folder Already Decrypted Successfully.")

    decrypt_button = tk.Button(folder_window,text="Decrypt Folder", font=("Arial", 14, "bold"),bg="red", fg="white",command=decrypt_folder)

    # arrange the widgets using grid
    folderpath_label.grid(row=0, column=0, sticky=tk.E)
    folderpath_entry.grid(row=0, column=1)
    browse_button.grid(row=0, column=2)
    password_label.grid(row=1,column=0)
    password_entry.grid(row=1,column=1)
    decrypt_button.grid(row=2, column=1)

    # center the widgets
    folder_window.grid_rowconfigure(0, weight=0)
    folder_window.grid_columnconfigure(1, weight=0)
    folderpath_label.grid(padx=10,sticky="e")
    folderpath_entry.grid(pady=10,sticky="w")
    browse_button.grid(padx=10,pady=2, sticky="w")
    password_label.grid(padx=10,sticky="e")
    password_entry.grid(pady=10,sticky="w")
    decrypt_button.grid(padx=100,sticky="w")

    # prevent the folder window from being destroyed
    folder_window.grab_set()

info_button = tk.Button(root, text="Project Info",font=("Arial", 14, "bold"),bg="red",fg="white",command=project_info)
info_button.pack(pady=20)
project_label = tk.Label(root,text="Folder Encrypt And Decrpyt!!!",font=("Arial", 18,"bold"),bg="black",fg="white")
project_label.pack(pady=25)

# Set base64 string image as the background
background_image_data = base64.b64decode(background_image_base64)
background_image = Image.open(io.BytesIO(background_image_data))
background_image = background_image.resize((200, 150), Image.ANTIALIAS)
background_photo = ImageTk.PhotoImage(background_image)
background_label = tk.Label(root, image=background_photo)
background_label.pack()

button1 = tk.Button(button_frame, text="Encrypt folders", font=("Arial", 14, "bold"), padx=10, pady=5, command=button1_clicked, bg="red", fg="white")
button2 = tk.Button(button_frame, text="Decrpyt folders",font=("Arial", 14, "bold"),padx=10, pady=5,command=button2_clicked,bg="red",fg="white")
# Pack the buttons in the frame

button1.pack(side="top", fill="x", padx=50, pady=10)
button2.pack(side="bottom", fill="x", padx=50, pady=10)

# Pack the frame in the root window
button_frame.pack(expand=True)


# Start the main event loop
root.mainloop()


C:\Users\vishn\AppData\Local\Temp\ipykernel_2396\1154515790.py:449: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  background_image = background_image.resize((200, 150), Image.ANTIALIAS)
